# Overview
This guide is to help first time users set up a Google Cloud Platform account specifically with the intention to use [tensorflow_cloud](https://github.com/tensorflow/cloud) to easily run training at scale on Google Cloud AI Platform. [tensorflow_cloud](https://github.com/tensorflow/cloud) provides APIs that allow users to easily go from debugging, training, tuning Keras and TensorFlow code in a local or kaggle environment to distributed training/tuning on Cloud.

<table align="left">
    <td>
        <a href="https://colab.research.google.com/github/tensorflow/cloud/blob/master/src/python/tensorflow_cloud/examples/google_cloud_project_setup_instructions.ipynb">
            <img width="50" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">Run in Colab
        </a>
    </td>
    <td>
        <a href="https://github.com/tensorflow/cloud/blob/master/src/python/tensorflow_cloud/examples/google_cloud_project_setup_instructions.ipynb">
            <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">View on GitHub
        </a>
     </td>
    <td>
        <a href="https://www.kaggle.com/nitric/google-cloud-project-setup-instructions">
            <img width="90" src="https://www.kaggle.com/static/images/site-logo.png" alt="Kaggle logo">Run in Kaggle
        </a>
     </td>
</table>


## 1. Sign up for Google Cloud Platform

To start go to https://cloud.google.com/ and click on “Get Started For Free". This is a two step sign up process where you will need to provide your name, address and a credit card. The starter account is free and it comes with $300 credit that you can use. For this step you will need to provide a Google Account ( i.e. your Gmail account) to sign in.

After completing the sign up process you will be redirected to [Google Cloud Platform welcome page](https://console.cloud.google.com/home/dashboard). click on the "Home" tab and make a note of your Project ID.

## 2.Enable Google Cloud SDK and Authenticate your notebook
Now that you have set up your project we can continue with the rest of the configuration steps directly from this notebook. There Three types of notebook, this step (step #2) is slightly different for each notebook, the rest of the steps (step #3 to #6) are the same for all notebooks.

* 2.1. Auth for Kaggle notebooks
* 2.2. Auth for Colab notebook
* 2.3. Auth for Cloud AI Notebooks - No action needed move to step 3.

### 2.1 Setup Auth for a Kaggle Notebook

If you are using a Kaggle notebook you will need to repeat this step for any new notebook that you use with this project. In your notebook click on Add-ons -> Google Cloud SDK and follow the instructions on the prompt. Then run the following command to add your auth credentials to this notebook.

In [ ]:
GCP_PROJECT_ID = 'YOUR_PROJECT_ID'

import sys
if "kaggle_secrets" in sys.modules:
    from kaggle_secrets import UserSecretsClient
    UserSecretsClient().set_gcloud_credentials(project=GCP_PROJECT_ID)

### 2.2 Setup Auth for a Colab Notebook
If you are using a Colab notebook you will need to repeat this step for any new notebook that you use with this project. Run the following command to add your auth credentials to this notebook.

In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## 3. Link your billing account to your project
Next step is to [set up  the billing account](https://cloud.google.com/billing/docs/how-to/modify-project) for this project. Google Cloud Creates a project for you by default which is called “My First Project”. We will use this default project. Use your Project ID (from step 1) to run the following commands. This will show you your Billing Account_ID, make a note of this for the next step.

In [ ]:
!gcloud iam service-accounts create YOUR_SERVICE_ACCOUNT_NAME


/bin/sh: line 1: gcloud: command not found


Use your Billing Account_ID from above and run the following to link your billing account with your project.

In [ ]:
!gcloud beta billing projects link 'YOUR_PROJECT_ID' --billing-account 'YOUR_BILLING_ACCOUNT_ID'

## 4. Enable Required APIs for tensorflow-cloud in your project
For tensorflow_cloud we use two specific APIs: [AI Platform Training Jobs API](https://console.cloud.google.com/marketplace/product/google/ml.googleapis.com) and [Cloud builder API]( https://console.developers.google.com/apis/api/cloudbuild.googleapis.com/overview). Note that this is a one time setup for this project, you do not need to rerun this command for every notebook.


In [ ]:
!gcloud services enable ml.googleapis.com cloudbuild.googleapis.com

## 5. Create a Google Cloud Storage bucket

We will use this storage bucket for temporary assets as well as to save the model checkpoints. Make a note of the name of the bucket for future reference. Note bucket names are unique globally.

In [ ]:
!gsutil mb gs://YOUR_BUCKET_NAME

## Create a service account for HP Tuning jobs
This step is required to use HP Tuning on Google Cloud using CloudTuner.
To [create a service account](https://cloud.google.com/iam/docs/creating-managing-service-accounts#iam-service-accounts-create-gcloud) run the following command to create a service account and make a note of it.

In [ ]:
!gcloud iam service-accounts create YOUR_SERVICE_ACCOUNT_NAME

Give permission to your service account to access Google Cloud Services by running the following command

In [ ]:
!gcloud projects add-iam-policy-binding YOUR_PROJECT_ID \
    --member serviceAccount:YOUR_SERVICE_ACCOUNT_NAME@YOUR_PROJECT_ID.iam.gserviceaccount.com \
    --role 'roles/editor'

In [ ]:
Finally run the following to allow your users account to impersonate the service account,

In [ ]:
!gcloud iam service-accounts add-iam-policy-binding \
    YOUR_SERVICE_ACCOUNT_NAME@YOUR_PROJECT_ID.iam.gserviceaccount.com \
    --member="user:YOUR_EMAIL_ADDRESS" \
    --role="roles/iam.serviceAccountUser"

## Congratulations !
You are now ready to run tensorflow-cloud. Note that these steps only need to be run one time. Once you have your project setup you can reuse the same project and bucket configuration for future runs. For any new notebooks you will need to repeat the step two to add your Google Cloud auth credentials. 